#### -*- coding: utf-8 -*-
"""
Created on Mon Sep  7 15:28:00 2020

@author: Frank
"""

In [2]:
from pyspark.sql import SparkSession
from pyspark.sql import functions as func
from pyspark.sql.types import StructType, StructField, IntegerType, LongType
import codecs

In [18]:
def loadMovieNames():
    movieNames = {}
    # CHANGE THIS TO THE PATH TO YOUR u.ITEM FILE:
    with codecs.open("/home/jovyan/work/Materials/u.item", "r", encoding='ISO-8859-1', errors='ignore') as f:
        for line in f:
            fields = line.split('|')
            movieNames[int(fields[0])] = fields[1]
    return movieNames

In [20]:
spark = SparkSession.builder.appName("PopularMovies").getOrCreate()

In [21]:
nameDict = spark.sparkContext.broadcast(loadMovieNames())

Create schema when reading u.data

In [35]:
print(len(list(nameDict.value.keys())))

1682


In [36]:
schema = StructType([ \
                     StructField("userID", IntegerType(), True), \
                     StructField("movieID", IntegerType(), True), \
                     StructField("rating", IntegerType(), True), \
                     StructField("timestamp", LongType(), True)])

Load up movie data as dataframe

In [42]:
moviesDF = spark.read.option("sep", "\t").schema(schema).csv("file:///home/jovyan/work/Materials/u.data")

In [43]:
moviesDF.show()

+------+-------+------+---------+
|userID|movieID|rating|timestamp|
+------+-------+------+---------+
|   196|    242|     3|881250949|
|   186|    302|     3|891717742|
|    22|    377|     1|878887116|
|   244|     51|     2|880606923|
|   166|    346|     1|886397596|
|   298|    474|     4|884182806|
|   115|    265|     2|881171488|
|   253|    465|     5|891628467|
|   305|    451|     3|886324817|
|     6|     86|     3|883603013|
|    62|    257|     2|879372434|
|   286|   1014|     5|879781125|
|   200|    222|     5|876042340|
|   210|     40|     3|891035994|
|   224|     29|     3|888104457|
|   303|    785|     3|879485318|
|   122|    387|     5|879270459|
|   194|    274|     2|879539794|
|   291|   1042|     4|874834944|
|   234|   1184|     2|892079237|
+------+-------+------+---------+
only showing top 20 rows



In [44]:
moviesDF.printSchema()

root
 |-- userID: integer (nullable = true)
 |-- movieID: integer (nullable = true)
 |-- rating: integer (nullable = true)
 |-- timestamp: long (nullable = true)



In [45]:
movieCounts = moviesDF.groupBy("movieID").count()

In [52]:
movieCounts.orderBy("movieID").show()

+-------+-----+
|movieID|count|
+-------+-----+
|      1|  452|
|      2|  131|
|      3|   90|
|      4|  209|
|      5|   86|
|      6|   26|
|      7|  392|
|      8|  219|
|      9|  299|
|     10|   89|
|     11|  236|
|     12|  267|
|     13|  184|
|     14|  183|
|     15|  293|
|     16|   39|
|     17|   92|
|     18|   10|
|     19|   69|
|     20|   72|
+-------+-----+
only showing top 20 rows



Create a user-defined function to look up movie names from our broadcasted dictionary

In [ ]:
def lookupName(movieID):
    return nameDict.value[movieID]

In [ ]:
lookupNameUDF = func.udf(lookupName)

Add a movieTitle column using our new udf

In [ ]:
moviesWithNames = movieCounts.withColumn("movieTitle", lookupNameUDF(func.col("movieID")))

Sort the results

In [ ]:
sortedMoviesWithNames = moviesWithNames.orderBy(func.desc("count"))

Grab the top 10

In [ ]:
sortedMoviesWithNames.show(10, False)

Stop the session

In [ ]:
spark.stop()